<center><img src="https://i.imgur.com/YNtjmbq.png" width="800"></center> 

# Práctica 2 – Exploración de Redes


En el año 2018, Quinten Dol sugirió a los usuarios de Twitter seguir a estas celebridades de la Tecnologia local:

In [2]:
from IPython.display import IFrame
IFrame('https://www.builtinseattle.com/2018/08/06/50-seattle-tech-twitter-accounts-to-follow', 800, 350)

Aunque proponía 50 _influencers_, sólo hay 46 activos en X (antes Twitter). Se construyó una red para ver si entre ellos se seguían, y esto tenemos:

In [5]:
IFrame('https://docs.google.com/spreadsheets/d/e/2PACX-1vQaaNQb2frjnlt0fPSBQhp5tNydB0a9ibp98sblBn6OfvGd8vsx8cfHb4w8Cm_qC3EIeYFby_zNQBoK/pubhtml', 800, 350)



 Los datos puedes descargarlos desde este enlace en tu carpeta **data** desde este [enlace](https://drive.google.com/drive/folders/1QXy0RwgNUBryirnVx0-_-rfr3WZxSld5?usp=sharing). No olvide indicar que **data** es el folder por defecto. Abramos en UCINET la matriz de adyacencia como en la sesión anterior (usando el **Matrix Editor**):
 
 
<img src="https://i.imgur.com/5CcgRfx.png" width="800"/>

La matriz de adyacencia se abrirá en el editor. Guardela en el formato de UCINET:


<img src="https://i.imgur.com/5RIrcza.png" width="800"/>

A esta altura, ya debes tener un archivo de UCINET con esta red.


<img src="https://i.imgur.com/2lZCLEg.png" width="800"/>


Ahora, como tenemos atributos, usemos el **DL editor** para abrirlo:



<img src="https://i.imgur.com/n9pKeC2.png" width="800"/>

Una vez abierto, guardalo como archivo UCINET también, pero indica que es de atributos en el nombre del archivo. Nota que estamos usando la opción _Matrix_ en el formato de datos:



<img src="https://i.imgur.com/7RR7Igy.png" width="800"/>


# Visualizando en NETDRAW

Como se sabe, UCINET usa NetDraw para dibujar. Como tenemos archivo de RED y de ATRIBUTO debemos seguir  pasos particulares en cada caso:

<img src="https://i.imgur.com/pt1QwcC.png" width="800"/>


1) La red se abre con la carpeta que muestra una _U_. Proceda.
2) Los atributos se abren con la carpeta que muestra una _A_. Proceda.

Si siguió los pasos, debe ver algo así:


<img src="https://i.imgur.com/fMZeC6c.png" width="800"/>






In [ ]:
<img src="https://i.imgur.com/mGnEjxg.png" width="800"/>